## Основные характеристики датасета

sepal length - длина наружной доли околоцветника

sepal width - ширина наружной доли околоцветника

petal length - длина внутренней доли околоцветника

petal width - ширина внутренней доли околоцветника

target - тип ирисов (Iris setosa, Iris virginica, Iris versicolor)

# Импорт библиотек

In [21]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, LeaveOneOut, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Подготовка

In [22]:
iris = load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [23]:
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [24]:
data.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

In [25]:
data.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [26]:
from sklearn.preprocessing import MinMaxScaler

mmScaler = MinMaxScaler()

scaled_data = mmScaler.fit_transform(data)
scaled_data = pd.DataFrame(scaled_data, columns=data.columns)
scaled_data['target'] = data['target']

In [27]:
scaled_data.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,0.428704,0.440556,0.467458,0.458056,1.000000
std,0.230018,0.181611,0.299203,0.317599,0.819232
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.333333,0.101695,0.083333,0.000000
50%,0.416667,0.416667,0.567797,0.500000,1.000000
75%,0.583333,0.541667,0.694915,0.708333,2.000000
max,1.000000,1.000000,1.000000,1.000000,2.000000


In [28]:
data = scaled_data

Пропусков в датасете нет.

Категориальный признак уже закодирован.

# Разделение на выборки

In [29]:
X = data.iloc[:, :-1]
y = data.target

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=145)

# Обучение с константным K

In [31]:
K = 4
knn = KNeighborsClassifier(n_neighbors=K)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [32]:
y_pred = knn.predict(X_test)

In [33]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9777777777777777


In [34]:
confusion_matrix(y_test, y_pred)

array([[14,  0,  0],
       [ 0, 17,  1],
       [ 0,  0, 13]], dtype=int64)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        14
         1.0       1.00      0.94      0.97        18
         2.0       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



# Подбор гиперпараметра K

In [36]:
param_grid = {'n_neighbors': np.arange(1, 31)}

In [37]:
skf = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=skf, scoring='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=125, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
             scoring='accuracy')

In [38]:
print("Лучший результат GridSearchCV: {:.3f} с K={}".format(grid_search.best_score_, grid_search.best_params_['n_neighbors']))

Лучший результат GridSearchCV: 0.973 с K=6


In [39]:
loo = LeaveOneOut()
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_grid, n_iter=20, cv=loo, scoring='accuracy', random_state=RANDOM_STATE)
random_search.fit(X, y)

RandomizedSearchCV(cv=LeaveOneOut(), estimator=KNeighborsClassifier(),
                   n_iter=20,
                   param_distributions={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                   random_state=125, scoring='accuracy')

In [40]:
print("Лучший результат RandomizedSearchCV: {:.3f} с K={}".format(random_search.best_score_, random_search.best_params_['n_neighbors']))

Лучший результат RandomizedSearchCV: 0.967 с K=7


# Сравнение

**Исходная** модель была обучена на 70% датасета со случайно выбраным K=4 и показала точность 0.9(5).

Для второй модели коэффициент подбирался при помощи кросс-валидации **StratifiedKFold** для сохранения соотношения классов и **GridSearchCV**. Был подобран K=5 с точностью 0.973. Был подобран более оптимальный коэффициент, и получена более высокая точность.

Для 3 модели использовалась кросс-валидация **LeaveOneOut** c **RandomizedSearchCv** для компенсации большого времени выполнения. Получена точность 0.98 при K=19. Разница в точности с предыдующими моделями мала и может быть снова объяснена оценкой точности. Также возможно, LOO обеспечила максимальное использование данных, что при малом размере датасета (150 строк) привело к лучшему результату